In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader

model = models.resnet50(weights=None)  # 가중치 없이 모델 구조만 정의
model.fc = nn.Linear(model.fc.in_features, 2)  # 저장할 때와 동일한 구조 설정
model = torch.load('model_v3.pt')  # 모델 불러오기

In [9]:
import os
from PIL import Image
from torchvision import transforms
import torch

directory = r'C:\Users\lwj01\HowFastTennisBallIs\novak_sinner_over_30'
save_directory = r'C:\Users\lwj01\HowFastTennisBallIs\y_v3'  # 저장할 경로

# CUDA 사용 가능 여부 확인 및 모델을 GPU로 이동
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
model = model.to(device)

def sliding_window(image, step_size, window_size):
    """이미지 위에서 슬라이딩 윈도우를 생성하는 함수"""
    for y in range(0, image.size[1] - window_size[1], step_size):
        for x in range(0, image.size[0] - window_size[0], step_size):
            yield (x, y, image.crop((x, y, x + window_size[0], y + window_size[1])))

def detect_objects_per_image(model, image_path, window_size=(30, 30), step_size=10):
    """하나의 이미지에 대해 객체를 탐지하는 함수"""
    model.eval()  # 모델을 평가 모드로 설정
    transform = transforms.Compose([transforms.Resize(window_size), transforms.ToTensor()])

    best_score = 0
    best_window = None
    best_location = None

    image = Image.open(image_path).convert('RGB')
    for (x, y, window) in sliding_window(image, step_size, window_size):
        window_tensor = transform(window).unsqueeze(0).to(device)  # 모델에 입력하기 위해 차원 추가
        output = model(window_tensor)
        score = torch.sigmoid(output).max().item()

        if score > best_score:
            best_score = score
            best_window = window
            best_location = (x, y)

    return best_score, best_location, best_window

# 모든 이미지에 대해 객체 탐지 실행
for filename in os.listdir(directory):
    if filename.endswith('.jpg'):
        image_path = os.path.join(directory, filename)
        score, location, window = detect_objects_per_image(model, image_path)
        print(f"Image: {filename}, Best Score: {score}, Location: {location}")
        if window is not None:
            save_path = os.path.join(save_directory, f"detected_{filename}")
            window.save(save_path)  # 윈도우 이미지를 지정된 경로에 저장


Using device: cuda:0
Image: frame_000000.jpg, Best Score: 0.9984367489814758, Location: (30, 10)
Image: frame_000001.jpg, Best Score: 0.9984362721443176, Location: (30, 10)


KeyboardInterrupt: 